In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from datetime import datetime

from sklearn.model_selection import train_test_split

In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('.'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

.\automl-getting-started-notebook.ipynb
.\sample_submission.csv
.\test.csv
.\train.csv
.\.ipynb_checkpoints\automl-getting-started-notebook-checkpoint.ipynb


In [3]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')

In [7]:
train.sample(6)

,id,keyword,location,text,target
4619,6567,injury,NaN,FollowNFLNews: Michael Floyd's hand injury sho...,0
737,1067,bleeding,Madisonville TN,@chaosmagician97 awesome!! I saw he was bleedi...,0
1141,1645,bombing,NaN,@NBCNews Yea bombing #pearlharbor not so good ...,1
1803,2591,crash,NaN,Photoshop CS6 Crash Course http://t.co/cVGJFPB...,0
6305,9008,stretcher,NaN,@PLlolz @Grazed @Stretcher @invalid @witter @T...,0
6398,9144,suicide%20bomber,NaN,Six people were killed Thursday when a Taliban...,1


### Data spelunking
How often does 'fire' come up in this dataset?

In [8]:
train.loc[train['text'].str.contains('fire', na=False, case=False)]

,id,keyword,location,text,target
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1
5,8,NaN,NaN,#RockyFire Update => California Hwy. 20 closed...,1
7,13,NaN,NaN,I'm on top of the hill and I can see a fire in...,1
...,...,...,...,...,...
7427,10625,wounded,NaN,Officer wounded suspect killed in exchange of ...,1
7433,10631,wounded,Yogya Berhati Nyaman,@wocowae Officer Wounded Suspect Killed in Exc...,1
7604,10863,NaN,NaN,#WorldNews Fallen powerlines on G:link tram: U...,1
7609,10870,NaN,NaN,@aria_ahrary @TheTawniest The out of control w...,1


Does the presence of the word 'fire' help determine whether the tweets here are real or false?

In [9]:
train.loc[train['text'].str.contains('fire', na=False, case=False)].target.value_counts()

1    344
0    129
Name: target, dtype: int64

In [20]:
train[train["target"] == 1]["text"].values[0:5]

array(['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
       'Forest fire near La Ronge Sask. Canada',
       "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
       '13,000 people receive #wildfires evacuation orders in California ',
       'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school '],
      dtype=object)

# CountVectorizer 

In [11]:
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
cv = feature_extraction.text.CountVectorizer()

In [21]:
cv_train=cv.fit_transform(train['text'])

In [22]:
cv_train.shape

(7613, 21637)

In [23]:
cv_train

<7613x21637 sparse matrix of type '<class 'numpy.int64'>'
	with 111497 stored elements in Compressed Sparse Row format>

In [30]:
cv_train[0].todense()

matrix([[0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [38]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
X=cv_train
y=train['target']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

lr=LogisticRegression()

In [39]:
lr=lr.fit(X_train,y_train)

In [41]:
lr_pred=lr.predict(X_test)

In [42]:
from sklearn.metrics import confusion_matrix,classification_report
print(confusion_matrix(lr_pred,y_test))
print(classification_report(lr_pred,y_test))

[[1266  314]
 [ 180  753]]
              precision    recall  f1-score   support

           0       0.88      0.80      0.84      1580
           1       0.71      0.81      0.75       933

   micro avg       0.80      0.80      0.80      2513
   macro avg       0.79      0.80      0.79      2513
weighted avg       0.81      0.80      0.81      2513



In [44]:
from sklearn.metrics import accuracy_score
print(accuracy_score(lr_pred,y_test)*100)

80.34222045364106


# Cross validation

In [46]:
scores = model_selection.cross_val_score(lr, cv_train, train["target"], cv=3, scoring="f1")
scores

C:\Users\akashmas\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


array([0.63952866, 0.61386139, 0.6848095 ])